In [4]:
import sys
import os
import pandas as pd
###There is something you need to change before using
# change the path where jupyter access
Path=r'C:\Users\Simu\Desktop\BRISTOL\DSMP'
#where change the path of data in "additional folder"
path_add_folder=r'C:\Users\Simu\Desktop\BRISTOL\DSMP\925 Data additional data'
#where change the path of data in "Rise data"
path_rise_folder=r'C:\Users\Simu\Desktop\BRISTOL\DSMP\Ceara Rise data'
sys.path.append(Path)
#merge name of excel into list files
files_add = [file for file in os.listdir(path_add_folder) 
             if file.endswith('.csv') or file.endswith('.xlsx')]
files_rise = [file for file in os.listdir(path_rise_folder) 
             if file.endswith('.csv') or file.endswith('.xlsx')]
def read_all_excel_from_folder(path,files):
    #initialize a library to storage
    list=[]
    for file in files:
        #join the path of excel to the path of folder
        file_path = os.path.join(path,file)
        #get data with panda
        if file.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)
        #get name of excel without exthension
        file_name_without_extension = os.path.splitext(file)[0]
        #storage data in library with name of excel to index
        list.append(df)
    return list

##storage data from additon_folder
excels_add= read_all_excel_from_folder(path_add_folder,files_add)
#example of indexing
print(excels_add[0])
#because the name of excel is complex so we read the name of file with number
#but it is ok to index with(excel_add[files_add.index('name')])
print(excels_add[files_add.index(files_add[0])])

##storage fata from rise_folder
excels_rise= read_all_excel_from_folder(path_rise_folder,files_rise)
print(excels_rise[0])


      Unnamed: 0  Object.ID    Area..µm².  Mean..Gray.Intensity.Value.  \
0              1         10  71960.336636                   115.860406   
1              2         11  71012.345738                   102.445560   
2              3         12  43433.638042                   139.667000   
3              4         13  47138.629994                   115.107011   
4              5         14  48938.942984                   141.558024   
...          ...        ...           ...                          ...   
2154        2155       7211  39328.576537                    97.571650   
2155        2156       7272  28596.275908                   120.901156   
2156        2157       7287  64680.810196                   132.147640   
2157        2158       7288  45233.951033                   123.688137   
2158        2159       7629  79779.087257                   166.323558   

      Shape.Factor  Min..Diameter...µm.  Max..Diameter...µm.  \
0         0.662955           246.498520        

In [3]:
pip install openpyxl


   ---------------------------------------- 250.9/250.9 kB 7.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


    ***I will change name of columns to make sure every excel have same columns , take out blank columns identified by panda and take out last three lines in excel of file"rise" because they are not data about specific samples(you can see why in details by opening excels in 'rise' file)
    add columns in excel which miss "area" and "Perimeter (µm)"
    rearrange columns by letters
    rename colimns
    add columns for indexing age

In [6]:
import re
#get value of attributes from filenames
def pre_filenames(filenames):
    name_attribute = []
    for filename in filenames:
        # take out extension and "measure ...."
        filename = filename.split(".")[0]
        filename = filename.replace("Count and Measure of", "")
        # get elements of number and letter
        elements = re.findall(r'\d+\.\d+|\d+|[a-zA-Z]', filename)
        # take out "-"
        parsed_elements=[]
        for element in elements:
            parsed_elements.extend(re.split(r'[-]', element))
        name_attribute.append(parsed_elements)
    return name_attribute
# function to reneme columns
def rename_columns(excels):
    new_columns = ['Area (µm²)', 
                   'Elongation', 
                   'Max (Diameter) (µm)',
                   'Mean (Diameter) (µm)', 
                   'Mean (Gray Intensity Value)',
                   'Min (Diameter) (µm)', 
                   'Object ID', 
                   'Perimeter (µm)', 
                   'Shape Factor',
                   'Sphericity']
    excels = [excel.rename(columns=dict(zip(excel.columns,new_columns))) for excel in excels]
    return excels
def add_columns(excels,name_lists):
    excels=[excel.assign(
        SITE = name[0],
        HOLE = name[1],
        CORE = name[2],
        CORE_TYPE=name[3],
        SECTION=name[4],
        TOP_DEPTH=name[5],
        BOTTOM_DEPTH=name[6]
        ) for excel,name in zip(excels,name_lists)]
    return excels

def drop_columns(excels,columns_name):
    return [df.drop(columns=[columns_name], errors='ignore') for df in excels]

#get name of files
pre_filenames_add=pre_filenames(files_add)
#print(pre_filenames_add)
pre_filenames_rise=pre_filenames(files_rise)


for i in range(len(excels_rise)):
    if excels_rise[i].shape[1]<excels_rise[0].shape[1]:
        excels_rise[i]["Area (µm²)"]=pd.NA
        excels_rise[i]["Perimeter (µm)"]=pd.NA
#test whether they have same number of columns
for i in range(len(excels_rise)):
    if excels_rise[i].shape[1]<excels_rise[0].shape[1]:
        print(excels_rise[i].columns)
        print(excels_rise[0].columns)
        print(i)
        
#drop unuseful column in "additional" excel
excels_add_name= drop_columns(excels_add, 'Unnamed: 0')
#drop unuseful column in  "rise" excel
excels_rise_name= drop_columns(excels_rise, 'Statistics')

#rearrange columns by letters
excels_add_name_re= [excel[sorted(excel.columns)] for excel in excels_add_name ]
excels_rise_name_re = [excel[sorted(excel.columns)] for excel in excels_rise_name]





#rename columns
excels_add_columns_rename= rename_columns(excels_add_name_re)

excels_rise_columns_rename= rename_columns(excels_rise_name_re)
excels_rise_columns_rename= [excel.iloc[:-3] for excel in excels_rise_columns_rename]

#add columns which is used to index age
pre_filenames_rise[253][-1]=84.5
pre_filenames_rise[253].append("86.5")
excels_add_columns_index= add_columns(excels_add_columns_rename,pre_filenames_add)
excels_rise_columns_index= add_columns(excels_rise_columns_rename,pre_filenames_rise)
print(excels_rise_columns_index[3])
excels_add_final = excels_add_columns_index
excels_rise_final = excels_rise_columns_index
print(excels_add_final[3].columns)
print(excels_rise_final[3].columns)
print(len(excels_add_final))
print(len(excels_rise_final))
for i in range(len(excels_add_final)):
    excels_add_final[i]['ID']=i
print(excels_add_final[1])
for j in range(len(excels_rise_final)):
    excels_rise_final[j]['ID']=j+len(excels_add_final)
print(excels_rise_final[-1]['ID'])

IndexError: list index out of range

    ***Merge excels in excels in 'rise' file and 'additional' file.
    Alse try to merge samples excels with 'Mastersheet'file to see whether it can add 'age' columns to samples excels.

In [ ]:
Master_sheet_stay=pd.read_excel(r"F:\DS Studying\mini project\data\925_Mastersheet.xlsx",engine='openpyxl')
Master_sheet =Master_sheet_stay.iloc[:-3]
# make suer type is same
def standardize_types(df):
    df["CORE"] = df["CORE"].astype(float)
    df["SITE"] = df["SITE"].astype(float)
    df["HOLE"] = df["HOLE"].astype(str)
    df["CORE_TYPE"] = df["CORE_TYPE"].astype(str)
    df["SECTION"] = df["SECTION"].astype(float)
    df["TOP_DEPTH"] = df["TOP_DEPTH"].astype(float)
    df["BOTTOM_DEPTH"] = df["BOTTOM_DEPTH"].astype(float)
    return df
print(Master_sheet.dtypes)
print(excels_rise_final[3].dtypes)
print(excels_add_final[2].dtypes)
Master_sheet = standardize_types(Master_sheet)
excels_rise_final_st = [standardize_types(df) for df in excels_rise_final]
excels_add_final_st = [standardize_types(df) for df in excels_add_final]
# drop lines where age is empty
key_columns = ["SITE", "HOLE", "CORE", "CORE_TYPE", "SECTION", "TOP_DEPTH", "BOTTOM_DEPTH", "Age (Ma)"]
key_index=["SITE", "HOLE", "CORE", "CORE_TYPE", "SECTION", "TOP_DEPTH", "BOTTOM_DEPTH"]
mastersheet_filtered = Master_sheet[key_columns].dropna(subset=["Age (Ma)"])
excels_rise_final_std = [excel_rise_final_st.dropna(subset=key_index) for excel_rise_final_st in excels_rise_final_st ]
excels_add_final_std = [excel_add_final_st.dropna(subset=key_index) for excel_add_final_st in excels_add_final_st] 
print(excels_rise_final_st[3])

# merge function which wil be used by map
def add_age_info(df):
    df = df.copy()  
    for col in ["HOLE", "CORE_TYPE"]:
        df[col] = df[col].astype(str)
    merged_df = df.merge(mastersheet_filtered, 
                         on=["SITE", "HOLE", "CORE", "CORE_TYPE", "SECTION", "TOP_DEPTH", "BOTTOM_DEPTH"], 
                         how="left")
    return merged_df

# storage with age
excels_add_final_updated = [add_age_info(df) for df in excels_add_final_st ]
print(excels_add_final_updated[-1]['ID'])
excels_rise_final_updated = [add_age_info(df) for df in excels_rise_final_st]
print(excels_rise_final_updated[-1]['ID'])
print(excels_rise_final_updated[0]['ID'])
#merge excels of samples
excel_add_whole = pd.concat(excels_add_final_updated,ignore_index=True,axis=0)
print(excel_add_whole['ID'].nunique())
excel_rise_whole =pd.concat(excels_rise_final_updated,ignore_index=True,axis=0)
print(excel_rise_whole['ID'].nunique())
samples=pd.concat([excel_add_whole,excel_rise_whole],ignore_index=True,axis=0)
print(samples['ID'].nunique())
samples.to_csv("samples.csv", index=False)

In [ ]:
samples["key"] = samples[["SITE", "HOLE", "CORE", "CORE_TYPE", "SECTION", "TOP_DEPTH", "BOTTOM_DEPTH"]].astype(str).agg("_".join, axis=1)
#set columns which used to group
group_cols = ["SITE", "HOLE", "CORE", "CORE_TYPE", "SECTION", "TOP_DEPTH", "BOTTOM_DEPTH"]
question_one_data = samples.groupby(group_cols).agg({
    "Max (Diameter) (µm)": lambda x: x.quantile(0.95),
    "Elongation": lambda x: x.quantile(0.95),
    "Age (Ma)": "first",  
    "Min (Diameter) (µm)": lambda x: x.quantile(0.05),
    "Shape Factor": "mean",
    "Sphericity":"mean"
    
}).reset_index()
question_one_data = question_one_data.copy().dropna(subset=["Age (Ma)",'Max (Diameter) (µm)','Elongation', "Sphericity", "Shape Factor","Min (Diameter) (µm)"])
print(question_one_data)
question_one_data.to_csv("question_one_data.csv", index=False)

In [ ]:
# Import necessary libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load your dataset (Ensure df_q1data_fixed is already cleaned)
df_q1data_fixed = pd.read_excel("q1_data")  # Uncomment if loading from a file

# Set plot style
sns.set(style="whitegrid")

# Create a figure for subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 18))

# 1. Scatter Plot: Max Diameter vs. Age (Million Years)
sns.scatterplot(
    data=df_q1data_fixed,
    x="Age_Ma",
    y="Max_Diameter_µm",
    hue="Age_Group",
    alpha=0.6,
    ax=axes[0]
)
axes[0].set_title("Scatter Plot: Max Diameter vs. Age")
axes[0].set_xlabel("Age (Ma)")
axes[0].set_ylabel("Max Diameter (µm)")

# 2. Boxplot: Distribution of Max Diameter over Age Groups
sns.boxplot(
    data=df_q1data_fixed,
    x="Age_Group",
    y="Max_Diameter_µm",
    ax=axes[1]
)
axes[1].set_title("Box Plot: Max Diameter Distribution Across Age Groups")
axes[1].set_xlabel("Age Group")
axes[1].set_ylabel("Max Diameter (µm)")

# 3. Line Plot: Mean Max Diameter Over Time
mean_size_over_time = df_q1data_fixed.groupby("Age_Ma")["Max_Diameter_µm"].mean().reset_index()
sns.lineplot(
    data=mean_size_over_time,
    x="Age_Ma",
    y="Max_Diameter_µm",
    marker="o",
    ax=axes[2]
)
axes[2].set_title("Line Plot: Mean Max Diameter Over Time")
axes[2].set_xlabel("Age (Ma)")
axes[2].set_ylabel("Mean Max Diameter (µm)")

# Adjust layout and show plots
plt.tight_layout()
plt.show()